# Возьмем готовую разбивку номенклатур по категориям

In [91]:
import pandas
import numpy as np
import codecs
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
colnames = ['name', 'id', 'class']
data = pandas.read_csv('nameCatIdCatName.csv', names=colnames, encoding='utf8')
names = data['name'].tolist()
classes = data['class'].tolist()
testLen = len(names) / 7
x_train = np.array(names[testLen:])
y_train = np.array(classes[testLen:])
x_test = np.array(names[:testLen]) 
y_test = np.array(classes[:testLen])

# Добавим две(sic!) строчки кода

In [92]:
pipeline = Pipeline([('vectorizer', TfidfVectorizer(ngram_range=(1,1))),('classifier',  MultinomialNB()) ])
pipeline.fit(x_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_id...      vocabulary=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [93]:
score = pipeline.score(x_test, y_test)
print(score)

0.71653250774


# Полученная точность 72%. Пробовал менять n-грамы, по-разному обрабатывать имена, но с наскоку больше не получилось

# Посмотрим, что все это значит

In [94]:
x_x = np.array([u'дезринит спрей назальный', u'acuvue линзы', u'garnier активный контроль', u'zewa платки', u'фарингосепт табл.'
                  , u'парацетамол 500', u'cosmos пластырь водостойкий', u'аспирин 500 табл', u'идеальная кожа чистая линия'])
predicts = pipeline.predict(x_x)
for x, p in zip(x_x, predicts):
    print(x + "  " + p)

дезринит спрей назальный  СРЕДСТВА ОТ ПРОСТУДЫ
acuvue линзы  ЛЕКАРСТВА ПО РЕЦЕПТАМ
garnier активный контроль  УХОД ЗА ЛИЦОМ
zewa платки  ГИГИЕНА
фарингосепт табл.  ЛЕКАРСТВА ПО РЕЦЕПТАМ
парацетамол 500  ТОВАРЫ ПЕРСОНАЛЬНОГО УХОДА
cosmos пластырь водостойкий  ПЕРВАЯ ПОМОЩЬ
аспирин 500 табл  ЛЕКАРСТВА ПО РЕЦЕПТАМ
идеальная кожа чистая линия  УХОД ЗА ЛИЦОМ


# Раз уж мы тут все собрались, то проверим, что будет с номенклатурами

In [44]:
content = codecs.open("nom3utf.csv", 'r', 'utf8')
real_test = np.random.choice(np.array(content.readlines()), size = 11)
predicts1 = pipeline.predict(real_test)
for x, p in zip(real_test, predicts1):
    print(x + "  " + p)

Бинт эласт PEHA-CREEP креп тк 4мх10см
  ПЕРВАЯ ПОМОЩЬ
Нео-Пенотран Форте супп. ваг. 750+200мг с напальч. бл. N7 Embil Pharm.
  ЛЕКАРСТВА ПО РЕЦЕПТАМ
ДИЗАО МАСКА ЖЕНЬШЕНЬ КОЛЛАГЕН ЭКСПРЕСС-УХОД ЗА КОЖЕЙ+ГИАЛУР КРЕМ N10 Jiang\su Bailan Group Incorporated Company
  УХОД ЗА ЛИЦОМ
Дрожжи пивные Экко Плюс с магнием табл. фл. N100 Экко Плюс
  БИОЛОГИЧЕСКИ-АКТИВНЫЕ ДОБАВКИ
МИРРОЛЛА масло вазелинов 25мл Мирролла
  УХОД ЗА ТЕЛОМ
Лизоретик табл. 20+12.5мг бл. N28 Ipca Laboratorie\s
  ЛЕКАРСТВА ПО РЕЦЕПТАМ
Ортофен табл. п.к/р.о. 25мг бл. N30 Мосхимфармпрепараты им.Н.А.Семашко
  ЛЕКАРСТВА ПО РЕЦЕПТАМ
Биовестин лакто эмульс. д/приема вн. 12мл фл. Био-Веста
  ЛЕКАРСТВА ПО РЕЦЕПТАМ
ПРЕЗИДЕНТ з/нить (фитофлосс) Сенситив 25м №1
  УХОД ЗА ПОЛОСТЬЮ РТА
Фервекс пор. д/р-ра д/приема вн. лим-сах 500+25+200мг 13.1г пак. N8 Bri\stol-Myer\s Squibb
  ЛЕКАРСТВА ПО РЕЦЕПТАМ
Масло д/укреплен волос 50мл Золотоношская ПКФ
  ЛЕЧЕБНАЯ КОСМЕТИКА


# Ну, не так уж и плохо